In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split


In [18]:
import json
import numpy as np
from sklearn.model_selection import train_test_split

strings = ["h_E_string","B_string","G_string","D_string","A_string","E_string"]

data = []
labels = []

for i, label in enumerate(strings):
    with open(f"{label}.json", "r") as file:
        json_data = json.load(file)
        values = json_data["optimal_bs"]
        data.append(values)
        labels.append(i)

# Pad the lists to the same length
max_length = max(len(lst) for lst in data)
data = np.array([lst + [0] * (max_length - len(lst)) for lst in data])

labels = np.array(labels)
# Prepare the data for the model
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)




In [22]:

model = Sequential([
    Dense(64, input_shape=(145,), activation='relu'),
    Dense(64, activation='relu'),
    Dense(6, activation='softmax')
])


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model

print(model.summary())
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │         9,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,894 (54.27 KB)

 Trainable params: 13,894 (54.27 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 866ms/step - accuracy: 0.0000e+00 - loss: 1.7927 - val_accuracy: 0.0000e+00 - val_loss: 1.7855
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 1.7848 - val_accuracy: 0.0000e+00 - val_loss: 1.7879
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 1.7782 - val_accuracy: 0.0000e+00 - val_loss: 1.7900
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 1.7721 - val_accuracy: 0.0000e+00 - val_loss: 1.7918
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 1.7660 - val_accuracy: 0.0000e+00 - val_loss: 1.7937
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 1.7601 - val_accuracy: 0.0000e+00 - val_loss: 1.7957
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 1.0000 - loss: 1.7545 - val_accuracy: 0.0000e+00 - val_loss: 1.7978
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 1.7490 - va

In [23]:
model.save('B_value_model.h5')